# MultAgent tests

In [82]:
import os
from dotenv import load_dotenv
load_dotenv()  

True

In [83]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [84]:
class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]

In [115]:
from langchain_core.messages import ToolMessage
import json

class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": outputs}

In [116]:
def route_tools(
    state: State,
):
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return END

In [117]:
from langchain_core.tools import tool
from langgraph.types import Command, interrupt
from datetime import datetime

@tool("create_transaction", parse_docstring=True)
def create_transaction(value: int, card_id: int, name: str) -> dict:
    """Criar uma nova transacao basica.
    
    Args:
        value: O valor da transacao
        card_id: id do cartao que o usuario quer registrar a transacao
        name: O nome que o usuario usou para identificar
    """
    return {
        'name': name,
        'value': value,
        'origem': card_id,
        'date': str(datetime.now())
    }

@tool("search_cards", parse_docstring=True)
def search_cards() -> int:
    """Buscar os cartoes com Id para outras ferramentas."""
    return [{
        'name': 'Cartao nubank',
        'type': 'Credit Card',
        'id': 345
    }]

@tool
def human_assistance(query: str) -> str:
    """Pedir algum dado faltante para o usuario."""
    print(query)
    human_response = interrupt({"query": query})
    return human_response["data"]

tools = [create_transaction, search_cards, human_assistance]

In [118]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("openai:gpt-4.1")
llm_with_tools = llm.bind_tools(tools)

In [120]:
def chatbot(state: State):
    llm_answer = llm_with_tools.invoke(state["messages"])
    return {"messages": [llm_answer]}

In [121]:
tool_node = BasicToolNode(tools=tools)

In [122]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

conn = sqlite3.connect("checkpoint.db", check_same_thread=False)

checkpointer = SqliteSaver(conn)

In [123]:
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph_builder.add_edge('tools', "chatbot")

graph_builder.add_conditional_edges(
    'chatbot',
    route_tools,
    {'tools': 'tools', END: END}
)

graph = graph_builder.compile(checkpointer=checkpointer)

In [126]:
config = {"configurable": {"thread_id": "4"}}
resp = graph.invoke(
    {"messages": [{"role": "user", "content": 'Qual cartao tenho?'}]},
    config
)
resp

{'messages': [HumanMessage(content='Qual cartao tenho?', additional_kwargs={}, response_metadata={}, id='b9d90573-99cc-4b1c-b15d-7850dc29ae95'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3fH5onJq6CMbh7tWPNvpiEyg', 'function': {'arguments': '{}', 'name': 'search_cards'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 133, 'total_tokens': 143, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_51e1070cf2', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-80c1e68c-bc22-4bed-a00f-effbe62bca81-0', tool_calls=[{'name': 'search_cards', 'args': {}, 'id': 'call_3fH5onJq6CMbh7tWPNvpiEyg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 133, 'output_tokens': 10, 'total_t

In [113]:
human_response = "O valor e 15 e o nome e uber"

human_command = Command(resume={"data": human_response})
final = graph.invoke(human_command, config, stream_mode="values")
final

Qual o valor e o nome ou descrição da transação que você deseja criar nesse cartão?
